# 法人籌碼動能策略

利用三大法人資金流向和技術分析指標相結合的方式，捕捉機構資金推動的股價上漲機會。

## 一、資料準備與處理

### 1. 資料導入與清洗
- 匯入套件、資料集：股價交易資訊、會計師簽證財務資料、集保庫存資料
- 統一資料日期格式，確保三個資料集可以按日期正確連接
- 僅保留TWSE(台灣證券交易所)和OTC(櫃買中心)的股票

In [ ]:
# 匯入套件、環境設定
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
# import setuptools
# import pandas_datareader.data as pdr   #DataReader API
import datetime as dt
# import yfinance as yf
# import mplfinance as mpf 
# from stocker2 import *  # 更改後的Stocker之視覺化套件
# from twstock2 import *
import talib
# from talib.abstract import *
# import requests
# from io import StringIO
from typing import List, Tuple, Dict
import alphalens as al
import vectorbt as vbt
import itertools
import os
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import warnings
warnings.filterwarnings("ignore")

# # TEJ API
# os.environ['TEJAPI_KEY'] = '2p6Pkl4B86cEaEgv8Bavt3HZxuhNRU'
# import tejapi
# tejapi.ApiConfig.api_base = 'https://api.tej.com.tw'
# tejapi.ApiConfig.api_key = '2p6Pkl4B86cEaEgv8Bavt3HZxuhNRU'

# Fubon API
# from fubon_neo.sdk import FubonSDK, Order   
# sdk = FubonSDK()
# accounts = sdk.login("L125278442", "Winnie891125", "C:/CAFubon/L125278442/L125278442.pfx", "Winnie891125")  # 需登入後，才能取得行情權限
# sdk.init_realtime() # 建立行情連線
# reststock = sdk.marketdata.rest_client.stock  
# reststock.intraday.tickers(#type='EQUITY', 
#                            #exchange="TWSE", 
#                            market='TSE',
#                            isNormal=True, 
#                            #industry='24'
#                            )


# # FinLab API
# import finlab
# finlab.login("u3C1OpNMI1362jt4csFYqZuqHiusvdzsWcuY3W8SnEX4CvNFf+d9iU8rD4L3ukkn")
# from finlab import data

# # FinMind API
# import FinMind
# from FinMind.data import DataLoader
# import requests
# url = "https://api.finmindtrade.com/api/v4/login"
# payload = {
#     "user_id": "pooh890320",
#     "password": "Winnie89@@",
# }
# data = requests.post(url, data=payload)
# data = data.json()
# api = DataLoader()
# api.login_by_token(api_token=data["token"])

## 處理中文亂碼
def plt_chinese():
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # 修改中文字體
    plt.rcParams['axes.unicode_minus'] = False # 顯示負號

plt_chinese()

# # # 獲取當前工作目錄
# # current_path = os.getcwd()
# # print(f"當前工作目錄：{current_path}")

## 匯入預先處理資料

# 股價交易資訊
df_trade = pd.read_csv("prcd_21_250415.csv", encoding="utf-8-sig")
df_trade['證券名稱'] = df_trade['證券名稱'].astype(str)
df_trade["資料日"] = pd.to_datetime(df_trade["資料日"])

# 會計師簽證財務資料
# df_fin = pd.read_csv("fin_21_250415.csv", encoding="utf-8-sig")

# 三大法人、融資券、當沖
df_shract = pd.read_csv("shract_21_250430.csv", encoding="utf-8-sig")
df_shract['證券名稱'] = df_shract['證券名稱'].astype(str)
df_shract["資料日"] = pd.to_datetime(df_shract["資料日"])
df_trade_shract = df_trade.merge(df_shract, on=['證券名稱','資料日','市場別'])  # 兩資料集格式相近，可整合在一起

- 調整係數63：cum return高
- 成交金額21,63：愈低愈好
- 當日均價21,63:愈低愈好
- 高低價差10,21,63:愈高愈好
- 個股市值63:愈低愈好
- 市值比重63:愈低愈好
- 股價淨值比5,10,21,63:愈低愈好
- 股價營收比5,10,21,63:愈低愈好
- 融資/券限額(千股)5,10:愈高愈好
- 融資維持率63:愈高愈好、cum return高
- 當沖維持率:第4組最好

### 2. 特徵工程
- **籌碼指標計算**：
  - 計算三大法人(外資、投信、自營商)每日買賣超張數
  - 計算法人5日累積買賣超張數 = 近5個交易日法人買賣超張數總和
  - 計算法人10日累積買賣超張數 = 近10個交易日法人買賣超張數總和
  - 計算法人5日買超強度 = 5日累積買賣超張數 / 流通在外股數
  - 計算法人持股比例變化率 = (當日持股率 - 5日前持股率) / 5日前持股率

- **價格動能指標計算**：
  - 計算5日移動平均線(周線，MA5)
  - 計算10日移動平均線(雙周線，MA10)
  - 計算21日移動平均線(月線，MA21)
  - 計算10日相對強弱指數(RSI10)
  - 計算股價與均線的相對位置指標(收盤價/MA10 - 1) × 100%
  - 計算成交量相對指標 = 當日成交量 / 21日平均成交量

In [50]:
### 計算周線、雙周線、月線、季線、年線 (talib.SMA)

def SMA_df(df):
    # 建立 pivot: index 為日期，columns 為股票，value 為收盤價
    price_wide = df.pivot(index='資料日', columns='證券名稱', values='收盤價')

    # 計算多種 SMA
    sma_df = vbt.talib('SMA').run(price_wide, timeperiod=[5, 10, 21, 63, 252])

    sma_df.real.index = pd.to_datetime(sma_df.real.index)

    return sma_df

    # 查詢:取出所有證券的某日某週期值 sma.real.xs(sma週期, level=0, axis=1).loc[某日期]
    # sma_df.real.xs(63, level=0, axis=1).loc["2025-01-03"]
    # df.sort_values(['證券名稱', '資料日']).groupby('證券名稱', group_keys=False)['收盤價'].transform(lambda x: SMA2.run(x, timeperiod=5))

### 計算某欄位的敘述統計，結果為寬表格
def pivot_summary(data: pd.DataFrame, col_name: str, window: int=0, summary_type: str=None):
    df_wide = data.pivot(index='資料日', columns='證券名稱', values=col_name)
    if summary_type is None:
        return df_wide
    elif summary_type == "mean":
        return df_wide.rolling(window).mean()
    elif summary_type == "sum":
        return df_wide.rolling(window).sum()
    else:
        return None

In [51]:
df_trade_shract[["外資買賣超張數","投信買賣超張數","自營買賣超張數(自行)","自營買賣超張數(避險)","合計買賣超張數","流通在外股數(千股)"]].head()

,外資買賣超張數,投信買賣超張數,自營買賣超張數(自行),自營買賣超張數(避險),合計買賣超張數,流通在外股數(千股)
0,-635.0,0.0,-587.0,-689.0,-1912.0,945500.0
1,-178.0,97.0,-257.0,-59.0,-397.0,945500.0
2,-1359.0,75.0,-689.0,-1280.0,-3253.0,945500.0
3,-1360.0,52.0,281.0,500.0,-527.0,944000.0
4,-7325.0,388.0,491.0,938.0,-5508.0,922000.0


In [52]:
### 5, 10日買賣超張數及買賣超強度

# 欲統計的欄位與視窗期間
columns_to_calc = ["外資買賣超張數","投信買賣超張數","自營買賣超張數(自行)","自營買賣超張數(避險)","合計買賣超張數"]
windows = [5, 10]

# 儲存結果的 dictionary
indicators = {}
outstanding_stock = pivot_summary(df_trade_shract, col_name="流通在外股數(千股)")

# 計算每個欄位的不同時間窗口累積量
for col in columns_to_calc:
    indicators[col] = pivot_summary(df_trade_shract, col_name=col)
    for win in windows:
        col2 = col.replace("買賣超張數", "")
        key = f"{col2}_{win}d"
        indicators[key] = pivot_summary(df_trade_shract, col_name=col, window=win, summary_type="sum")
        # 買賣超強度 = 5,10日累計 / 流通股數（千股）
        strength_key = f"{col2}_pressure_{win}d"
        indicators[strength_key] = indicators[f"{col2}_{win}d"] / outstanding_stock

    
# summary_results["外資_pressure_5d"].reset_index().melt(id_vars='資料日')  # wide to long


In [53]:
### 三大法人5日持股變化

for col in ["外資","投信","自營商"]:
    indicators[f"{col}5日持股變化"] = pivot_summary(df_trade_shract,col_name=f"{col}持股率").pct_change(5)
    
# df_trade_shract.sort_values(by=["資料日","證券名稱"]).groupby("證券名稱")[["外資持股率","投信持股率","自營商持股率"]].apply(lambda g: g.pct_change(5))#.reset_index(drop=False)

In [55]:
### SMA5, SMA10, SMA21, RSI10, 股價與ma10的相對位置, 成交量相對指標

indicators['close'] = df_trade_shract.pivot(index='資料日', columns='證券名稱', values='收盤價')
indicators['high'] = df_trade_shract.pivot(index='資料日', columns='證券名稱', values='最高價')
indicators['volume'] = df_trade_shract.pivot(index='資料日', columns='證券名稱', values='成交量(千股)')

indicators["RSI10"] = vbt.RSI.run(indicators['close'],10).rsi
for i in [5,10,21]:
    indicators[f"SMA{i}"] = SMA_df(df_trade_shract).real.xs(i, level=0, axis=1)

# indicators["ratio_close_SMA10"] = (indicators['close']/indicators["SMA10"]-1)#.applymap(lambda x: "{:.2%}".format(x))  # 股價與ma10的相對位置


indicators['volume'] = df_trade_shract.pivot(index="資料日", columns="證券名稱",values="成交量(千股)")
indicators["relative_volume"] = indicators['volume']/(indicators['volume'].rolling(21).mean())  # 成交量相對指標：當日/前21日平均

for dt in indicators.keys():
    indicators[dt] = indicators[dt].replace([np.inf, -np.inf], np.nan)

# 沒用到：外資_10d, 投信_10d, 自營(自行)_10d, 自營(避險)_10d, ratio_close_SMA10

## 二、選股階段（每日執行）

### 1. 初步籌碼篩選
- 篩選出三大法人5日買超且10日買超的股票
- **剔除三大法人買超但自營商大幅賣超的股票(通常代表風險規避)**
- 計算三大法人買超強度排名(買超張數/流通在外股數)

### 2. 價格動能確認
- 從籌碼篩選的結果中，進一步選出：
  - 股價站在5日和10日移動平均線之上的股票
  - RSI10指標介於50-80之間的股票(有上漲動能但未過熱)
  - 最近5日股價上漲且成交量逐漸放大的股票

### 3. 評分系統

建立綜合評分系統(0-100分)：
- 法人買超強度佔50分：
  - 5日買超強度(20分)
  - 10日買超強度(10分)
  - 方向一致性--同向買超(10分)
  - 合計買超趨勢(10分)
  - 持股變化(5分)
- 價量因子佔30分：
  - 本益比(10分)
  - 調整係數(10分)
  - 股價相對強弱/是否突破高點(10分)
- 成交量表現佔15分：
  - 成交量放大程度(10分)
  - 價量配合度(5分)
  - 成交金額
- 額外加減分
  - 外資投信賣超但自營避險買超（-15分)

### 4. 選股結果
  
- 按照評分高低排序股票
- 選出評分超過75分的股票作為候選池
- 控制候選池大小(如最多20檔股票)

In [57]:
indicators.keys()

dict_keys(['外資買賣超張數', '外資_5d', '外資_pressure_5d', '外資_10d', '外資_pressure_10d', '投信買賣超張數', '投信_5d', '投信_pressure_5d', '投信_10d', '投信_pressure_10d', '自營買賣超張數(自行)', '自營(自行)_5d', '自營(自行)_pressure_5d', '自營(自行)_10d', '自營(自行)_pressure_10d', '自營買賣超張數(避險)', '自營(避險)_5d', '自營(避險)_pressure_5d', '自營(避險)_10d', '自營(避險)_pressure_10d', '合計買賣超張數', '合計_5d', '合計_pressure_5d', '合計_10d', '合計_pressure_10d', '外資5日持股變化', '投信5日持股變化', '自營商5日持股變化', 'close', 'high', 'volume', 'RSI10', 'SMA5', 'SMA10', 'SMA21', 'relative_volume'])

In [58]:
#### 法人買超強度評分
### 5, 10日買超強度評分(佔30分)
# 權重1 = 外資:投信:自營(自行):自營(避險):合計 = 5:3:0.5:1:0.5
# 權重2 = 5日:10日 = 2:1
sources = {"外資":5, "投信":3, "自營(自行)":.5, "自營(避險)":1, "合計":.5}; days = {"5":2, "10":1}
scores = pd.DataFrame()

for source, day in itertools.product(sources.keys(), days.keys()):
    weight = sources[source]*days[day]
    scores[f"{source}_pressure_{day}d"] = indicators[f"{source}_pressure_{day}d"].apply(
        lambda row: (row.where(row > 0).rank(method="average") / row.where(row > 0).count() * weight).fillna(0),
          axis=1).reset_index().melt(id_vars='資料日').set_index(["證券名稱","資料日"])


In [59]:
### 方向一致性評分（佔10分）
# 外資、投信5日累積皆為買超（同向）--6分
# 外資、投信、自營(自行)5日累積皆為買超（同向）--4分

scores["方向一致性"] = (((indicators['外資_5d']>0) & (indicators['投信_5d']>0) )*6+\
                   ((indicators['外資_5d']>0) & (indicators['投信_5d']>0) & \
                    (indicators['自營(自行)_5d']>0) & (indicators['合計_5d']>0))*4).\
                      reset_index().melt(id_vars='資料日').set_index(["證券名稱","資料日"])

### 合計買超趨勢（佔10分）
# 連續買超得分：連續買超幾天就得幾分，最多5分

ss = pd.DataFrame(0, index=indicators['合計買賣超張數'].index, columns=indicators['合計買賣超張數'].columns)
for i in range(1,5+1):
    streak = indicators['合計買賣超張數'].gt(0).rolling(window=i, min_periods=i).sum() == i  
    # gt(0): 大於0為True，rolling((window=i, min_periods=i)).sum(): 連續True有多少，只會有0或i兩種值
    ss = ss.mask(streak, i)  # 只覆蓋 True 的地方為 i 分
    
# 5日累積買超>10日的一半，5分，若否但5日累積為買超得3分
ss = ss+(indicators['合計_5d'].gt(0)*3).mask(indicators['合計_5d']>(indicators['合計_10d']*0.5), 5)
scores['合計買超趨勢'] = ss.reset_index().melt(id_vars='資料日').set_index(["證券名稱","資料日"])

# 持股變化（佔5分）
# 外資5日持股變化介於0.3~2--得5分，0.2~0.3--得4分，0.1~0.2--得3分，0~0.1得1分
scores['持股變化'] = indicators['外資5日持股變化'].apply(lambda x: pd.cut(x, bins = [0,0.1,0.2,0.3,2], labels = [1,3,4,5], ordered=False), axis=1).\
    astype(float).fillna(0).reset_index().melt(id_vars='資料日').set_index(["證券名稱","資料日"])


In [60]:
#### 價格動能評分

### RSI表現（佔10分）
# RSI 40~45、80~85得3分，45~50得5分，50~60、70~80得9分，60~70得10分
rsii = indicators['RSI10'].apply(lambda x: pd.cut(x, bins = [40,45,50,60,70,80,85], labels = [3,5,9,10,8,3], ordered=False), axis=1).\
    astype(float).fillna(0)
rsii.columns = rsii.columns.get_level_values(1)
scores['RSI'] = rsii.reset_index().melt(id_vars='資料日').set_index(["證券名稱","資料日"])

In [61]:
### 均線多頭排列程度（佔10分）
# 收盤價 > SMA5 > SMA10 > SMA21 （完美多頭）得5分
# 收盤價 > SMA5 或 SMA10 得3分
# 收盤價 > SMA5 得1分

cond_1 = indicators['close']>indicators['SMA5']
cond_2 = cond_1 & (indicators['close']>indicators['SMA10'])
cond_3 = cond_1 & (indicators['SMA5']>indicators['SMA10']) & (indicators['SMA10']>indicators['SMA21'])#.reset_index().melt("資料日")

# SMA5今日 > SMA5昨日 得3分， 若滿足以上且SMA10今日 > SMA10昨日 得5分

cond_4 = indicators['SMA5'].diff(1).gt(0)
cond_5 = cond_4 & indicators['SMA10'].diff(1).gt(0)

scores['SMA_sort'] = ((cond_1*1).mask(cond_2, 3).mask(cond_3, 5)+(cond_4*3).mask(cond_5, 5)).reset_index().melt(id_vars='資料日').set_index(["證券名稱","資料日"])



In [62]:
### 股價相對強弱/是否突破高點（佔10分）
# 突破前10日高點--得10分，前5日高點--得7分，3日高點--得5分，突破前一日高點的3%--得4分，突破前一日高點--得2分
cond_1 = indicators["close"] > indicators['high'].shift(1)
cond_2 = indicators["close"] > (indicators['high'].shift(1))*1.03
cond_3 = indicators["close"] > indicators['high'].shift(1).rolling(window=3).max() 
cond_4 = indicators["close"] > indicators['high'].shift(1).rolling(window=5).max() 
cond_5 = indicators["close"] > indicators['high'].shift(1).rolling(window=10).max() 

scores['price_crosshigh'] = (cond_1*2).mask(cond_2, 4).mask(cond_3, 5).mask(cond_4, 7).mask(cond_5, 10).reset_index().melt(id_vars='資料日').set_index(["證券名稱","資料日"])


In [63]:
### 成交量表現

### 成交量放大程度（當日/近21日）（佔10分）
# 0~1得0分，1~1.2得1分，1.2~1.5得2分，1.5~1.8得4分，1.8~2.2得6分，2.2~3得8分，3以上得10分

scores['volume_enlarge'] = indicators['relative_volume'].apply(lambda x: pd.cut(x, bins = [0,1,1.2,1.5,1.8,2.2,3,1000], labels = [0,1,2,4,6,8,10], ordered=False), axis=1).\
    astype(float).fillna(0).reset_index().melt(id_vars='資料日').set_index(["證券名稱","資料日"])

### 價量配合度（佔5分）
# 收盤價>SMA5且成交量>5日平均--得5分，收盤價>SMA5且成交量>10日平均--得3分，收盤價<SMA5且成交量<10日平均--得2分
cond_1 = (indicators['close'] < indicators['SMA5']) & (indicators['volume'] <= indicators['volume'].rolling(window=10).mean())
cond_2 = (indicators['close'] > indicators['SMA5']) & (indicators['volume'] > indicators['volume'].rolling(window=10).mean())
cond_3 = (indicators['close'] > indicators['SMA5']) & (indicators['volume'] > indicators['volume'].rolling(window=5).mean())
scores['pvmatch'] = (cond_1*2).mask(cond_2, 3).mask(cond_3, 5).reset_index().melt(id_vars='資料日').set_index(["證券名稱","資料日"])


In [64]:
# 外資賣超 但 自營(避險)買超比例超過0.5%，代表短期跌機率高，扣15分
scores['risk_aversion'] = (((indicators['外資買賣超張數']<0)&\
  (indicators['自營買賣超張數(避險)']/outstanding_stock>0.005))*(-15)).reset_index().melt("資料日").set_index(["證券名稱","資料日"])
# 外資>0.5% and 自營<-0.1%

## 三、交易信號生成

### 1. 買入條件
設定精確的買入信號：
- 主要條件：三大法人連續3個交易日買超，且
- 次要條件：股價收在5日均線之上，且
- 強化條件：當日成交量大於前20日平均成交量1.2倍
- 排除條件：股價已連續上漲超過7天或單日漲幅超過5%

### 2. 賣出條件
設定明確的賣出信號：
- 停損條件：股價跌破10日移動平均線且下跌超過3%
- 獲利了結：股價漲幅達到15%或RSI超過80進入超買區
- 籌碼轉向：三大法人轉為連續2日賣超且單日賣超量大於過去5日平均買超量
- 趨勢轉折：MACD出現死亡交叉信號

In [87]:
aa = scores[['合計買超趨勢']].reset_index().pivot(index='資料日', columns='證券名稱', values='合計買超趨勢').ge(3)
bb = indicators['close']>indicators['SMA5']
aa, bb = aa.align(bb)

cc = (indicators['close']<indicators['SMA5']) & (indicators['close'].pct_change(1)<-0.03)

In [88]:
scores_sum = scores.iloc[:,13:].sum(axis=1)
# scores_sum.xs("2021-01-11", level=1).sort_values(ascending=False).index[:15].tolist()

def strategy_2(data=df_trade_shract, rebalance_date=5, stock_num=15, 
               start_date="2021-01-01", end_date="2025-04-15", rsi_entry=30, rsi_exit=70):

    init_capital=1_000_000
    df=data.copy()
    # sma_df=SMA_df(df)

    df['資料日'] = pd.to_datetime(df['資料日'])

    # 將收盤價轉成價格矩陣
    price = df.pivot(index='資料日', columns='證券名稱', values='收盤價').loc[start_date:end_date]
    # 將開盤價轉成價格矩陣
    # open = df.pivot(index='資料日', columns='證券名稱', values='開盤價').loc[start_date:end_date]
    # RSI 計算
    rsi = vbt.RSI.run(price, window=14)

    # 建立 rebalance 時點（每n天一次）
    rebalance_dates = price.index[::rebalance_date]

    # 建立 weight 矩陣
    weights = pd.DataFrame(0, index=price.index, columns=price.columns)

    for date in rebalance_dates:
    
        selected = scores_sum.xs(date, level=1).sort_values(ascending=False).index[:stock_num].tolist()

        if bool(selected) and len(selected)>0:

            # open_prices = open.loc[date, selected].dropna()
            # if open_prices.empty:
            #     continue

            # 根據前一天收盤價計算每支股票買入股數（股數 = 投入金額 / 前一天收盤價）
            # 這邊只要求投資組合的各股票比例（總和為1），因此只要根據股數與開盤價成反比撰寫即可 # 錯了，可以改
            # weights.loc[date, open_prices.index] = inv_open_prices / inv_open_prices.sum()  #投資比例（總和為1）
            
            weights.loc[date, selected] = 1 / len(selected)  # 每支股票持倉相同比例
            # print((weights.loc[date][weights.loc[date]!=0]))
        else:
            continue

    # 向下填滿，模擬持股直到下次rebalance
    weights = weights.replace(0, np.nan).ffill().fillna(0)
    target_shares = (weights * init_capital) / price.shift(1) # 權重（股數）為資金平分/前一天收盤價 

    # 產生 RSI 交易訊號
    entries = (rsi.rsi < rsi_entry) & (weights > 0)  # 滿足條件且在持股內才買進
    exits = (rsi.rsi > rsi_exit) & (weights > 0)   # RSI 大於 70 且有持倉就賣出
    short_entries = (rsi.rsi <25) & (weights > 0)  # 滿足條件且在持股內才買進
    short_exits = (rsi.rsi >75) & (weights > 0)   # RSI 大於 70 且有持倉就賣出

    # # 停損條件（價格低於進場後的某比例）
    # drawdown_thresh = 0.9  # 表示虧損超過 10%
    # entry_price = price.vbt.signals.first_occurrence(entries).ffill()
    # exit_stoploss = price < (entry_price * drawdown_thresh)

    # 執行回測
    pf = vbt.Portfolio.from_signals(
        close=price,
        entries=entries,#(aa&bb).loc[start_date:end_date],
        exits=cc.loc[start_date:end_date] & exits,
        # short_entries=short_entries,
        # short_exits=short_exits,
        size=target_shares,  # 每次買進固定"股數"，看起來from_signal只能平均每支股票的init_cash
        init_cash=init_capital,
        fees=0.001425,  # 手續費
        slippage=0.001,  # 滑價比例（實際成交價與理想間的差異）
        size_type="Amount",
        direction="longonly",
        freq='1D',
        cash_sharing=True,  # 啟用資金共享
        call_seq='auto',  # 多個操作同時出現時的處理順序（auto-自動決定；entry-進場；exit-出場；both-都要??）
        accumulate=True,  # 已有持倉的情況下是否累積（False-每次進場要先把舊的全部平倉）
        allow_partial=True, # 是否允許部分資金下單
        upon_opposite_entry='close',  # 遇到相反信號時關閉持倉
        sl_stop=0.5,  # 停損比例（跌超過多少比例就平倉）
        sl_trail=0.2,  # 移動停損（最高點回落多少就平倉停損）
        # tp_stop=0.5,

    )
    return pf

In [102]:
pf2 = strategy_2(end_date="2025-04-15", rebalance_date=21, rsi_entry=20, rsi_exit=80,stock_num=10)

# 顯示報酬結果
print(pf2.stats())

Start                         2021-01-04 00:00:00
End                           2025-04-15 00:00:00
Period                         1035 days 00:00:00
Start Value                             1000000.0
End Value                          2361826.906892
Total Return [%]                       136.182691
Benchmark Return [%]                    30.555792
Max Gross Exposure [%]                      100.0
Total Fees Paid                       9116.802048
Max Drawdown [%]                        53.275735
Max Drawdown Duration           692 days 00:00:00
Total Trades                                   26
Total Closed Trades                            10
Total Open Trades                              16
Open Trade PnL                      386335.070111
Win Rate [%]                                 40.0
Best Trade [%]                         448.474063
Worst Trade [%]                        -53.794573
Avg Winning Trade [%]                  128.359257
Avg Losing Trade [%]                   -33.057936


In [101]:
import plotly.graph_objects as go
fig = pf2.plot(subplots=['cum_returns'])  # pf1.plot_cum_returns()
fig.update_layout(yaxis_tickformat='.2%')  # 將 y 軸改為百分比格式
fig.show()

In [91]:
pf2.cash().vbt.plot(title='Portfolio Remaining Cash').show()

## 三、策略實施中的資料權重分配


### 2. 特殊情況處理
設置特殊條件篩選：

```
# 外資是否連續買超
外資連續買超天數 = 計算外資連續買超的天數

# 投信買超量創階段新高
投信買超階段高點 = 近30天投信單日買超量最大值
是否創新高 = 今日投信買超量 > 投信買超階段高點

# 自營商避險部位變化
自營商避險部位增減 = 自營商避險部位買賣超張數

# 特別關注點
if 外資連續買超天數 >= 3 and 投信5日累積買超 > 0:
    候選股優先級 += 2  # 提高優先級
    
if 外資5日累積買超 > 0 and 投信5日累積買超 > 0 and 自營商5日累積買超 < 0:
    # 自營商反向操作可能是對沖或技術性調整，不一定是看空
    # 外資+投信買超更重要
    pass
```

## 四、實際策略中的最佳實踐

### 2. 信號確認機制
- **強烈買入信號**：外資+投信同步買超，且三大法人5日、10日累積買超均為正
- **中等買入信號**：外資或投信之一持續買超，另一家不是大幅賣超
- **觀望信號**：法人買賣互見，方向不一致
- **賣出警示**：外資+投信同步轉為賣超

### 3. 特別觀察指標
- **法人買超佔成交量比例**：法人買超張數/當日成交量 > 20%更有意義
- **法人持股比例變化**：持股比例連續上升更能確認趨勢
- **買超集中度**：法人買超是否集中在特定幾天，還是平均分布

### 4. 回測建議
在使用vectorbt進行回測時：
- 嘗試不同的法人權重組合(如外資:投信:自營商 = 6:3:1或5:4:1)
- 測試不同的累積天數(3/5/7/10/15日)對結果的影響
- 分析在不同市場環境下(多頭/空頭/盤整)各法人指標的有效性

## 五、結論與建議

基於台股法人特性，建議採用**「分開計算、差異權重、合併評估」**的方法：

1. **分開計算**各類法人的買賣超數據和趨勢
2. **設定差異權重**，一般而言外資 > 投信 > 自營商
3. **關注一致性**，三大法人方向一致時的信號最強
4. **動態調整**，根據不同市場階段調整法人權重

這種處理方式能夠更全面地掌握市場資金流向，避免被單一法人的短期行為誤導，也能在回測中找出最優的參數組合，提高策略的穩定性和獲利能力。

## 四、倉位與風險管理

### 1. 資金分配
- 單一股票最多配置總資本的10%
- 同一產業股票總計最多配置30%
- 保留至少20%現金應對市場波動

### 2. 進場策略
- 分批買入：首次買入配置5%資金，信號加強時增加至10%
- 優先選擇評分最高的股票
- 法人買超強度較大的股票可考慮較大倉位

### 3. 風險控制
- 單支股票最大風險控制：設定10%止損點
- 投資組合整體風險：當投資組合回撤超過7%時，減少持倉50%
- 市場風險控制：大盤跌破季線時減倉至50%以下

## 五、持倉管理與調整

### 1. 定期評估
- 每日檢視持倉股票的籌碼變化
- 每週重新評估整體投資組合表現
- 每月重新調整選股標準和權重

### 2. 動態調整
- 績效跟踪：計算每支股票相對大盤的超額收益
- 倉位優化：表現較佳的股票可適度增加倉位(但不超過10%)
- 汰弱留強：替換表現不佳且籌碼轉弱的股票

### 3. 特殊情況處理
- 市場大幅波動時暫停新增部位
- 股票遇到重大利多/利空消息時特別評估是否調整持倉
- 財報季前後特別關注財報數據與法人動向的配合

## 六、回測與優化設計 (使用vectorbt)

### 1. 回測參數設定
- 回測區間：至少涵蓋3-5年數據，包含不同市場環境
- 手續費設定：買入0.1425%，賣出0.4425%(含證交稅)
- 滑點設定：0.1-0.3%，根據股票流動性調整

### 2. 策略優化方向
- 法人籌碼參數優化：測試不同累積天數(3/5/10/15日)的效果
- 動能指標參數調整：測試不同RSI周期與閾值
- 持倉時間優化：分析最佳持有天數和調倉頻率
- 止損點位優化：測試不同止損百分比的效果

### 3. 績效評估指標
- 年化收益率(與大盤比較)
- 夏普比率(風險調整後收益)
- 最大回撤幅度與持續時間
- 勝率與賺虧比
- 換手率與交易成本分析

## 七、實盤執行

### 1. 日常操作流程
- 盤前準備：更新資料、運行選股模型
- 盤中監控：追踪買入信號和賣出信號
- 盤後分析：記錄當日交易、評估策略表現、準備次日計劃

### 2. 交易記錄
建立詳細的交易日誌，記錄：
- 交易日期、股票代碼、買賣方向
- 交易價格、數量、總金額
- 交易原因(符合哪些信號)
- 當時的市場狀況與法人籌碼
- 交易後的自我評估

### 3. 策略調整機制
- 設定固定的策略複審期間(如每季一次)
- 當策略連續3個月跑輸大盤時進行全面檢討
- 根據市場環境變化調整籌碼與技術指標的權重

---

## 實現要點與注意事項

1. **資料質量保證**：
   - 確保三大法人資料的及時更新(通常有T+1的延遲)
   - 處理除權息對股價和成交量的影響
   - 剔除流動性不足的股票(如日均成交量低於300張)

2. **信號生成的穩健性**：
   - 避免頻繁交易，設定適當的信號確認機制
   - 考慮加入噪音過濾，如信號必須持續n天才確認
   - 結合市場整體氛圍判斷，避免逆大勢而行

3. **風險控制重點**：
   - 重視下檔保護多於上漲追求
   - 設定投資組合層面的風險預算
   - 觀察法人買超資金規模與持續性

4. **避免常見陷阱**：
   - 防止追高：設定買入價格上限(如不超過5日最高價)
   - 避免戀愛股票：嚴格執行賣出紀律
   - 警惕籌碼陷阱：區分主力作帳與真實法人買盤

這個策略適合中期持股(數週到數月)，注重跟隨法人資金流向並搭配技術面確認，適合具有基本量化分析能力的新手交易者。透過vectorbt的回測功能，您可以最佳化參數並檢視策略在不同市場環境下的表現。


- PCA可以讓眾多籌碼、價量因子降維，降維後再做因子分析也可
- FA形成的共同因子可以將其命名解釋，但因子太多了因子分數應該很複雜，暫不考慮FA
- FPCA只能針對其中一個因子"隨時間的曲線"降維，處理上可能要考慮切成多種時間點
- MFPCA可以處理多個因子，但目的不在於對因子降維，而是用少許的曲線重建資料或是用這些曲線分類，相較於FPCA的好處即是能考量因子間的共變異
    - 如果時間段太長，可能訊息太多也分的不夠好

In [ ]:
print("helloooooo")

# which git     #git在電腦內的路徑
# git -v/--version  #查看git版本
# ls    #查看專案中的檔案有啥
# ls 路徑   #確認有無此檔案路徑
# git init   #初始化專案
# git branch -M <名稱>    # 設定專案內的分支名稱

# git status    #檢查是否有未提交、未追蹤的檔案
# git add 檔案名稱      #新增檔案到追蹤範圍
# git add 資料夾       #直接新增該資料夾底下所有內容
# git reset     #重設追蹤內容
# git commit -m 'xxx'   #提交至git (xxx為自訂提交/修改說明)

# 做下面這行才是將git與github連結
# git remote add origin https://github.com/<您的使用者名稱>/<儲存庫名稱>.git    #連接遠端資料庫至github專案
# git remote -v     # 檢查遠端儲存庫狀態
# git remote remove  #移除遠端儲存庫
# git remote set-url <專案代碼> <github專案網址>
# git push #確定上傳（於目前分支）
# git push (-u) origin main   #上傳於main這個分支


helloooooo
